In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras import Model
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import class_weight

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [ ]:
tf.device('/device:GPU:2')

In [ ]:
data_df = pd.read_csv('label_preprocessed.csv')
data_df = data_df.drop(['URL'], axis=1)

In [ ]:
data_df

,ExpPeriod,Salary,Age,Gender,WorkType,WorkSchedule,N_places,LastWorkDesc,LastWorkDesc2,Top 10 work,...,Description,Python,SQL,Power BI,N_Unis,Top 10 Uni,top_city,EduLevelCat,DataAnalyst,DataScientist
0,-1.048106,-0.032825,-0.851000,2.0,1.0,0.0,-0.311920,работ проект image реcогнитионобеспечен полнот...,работ отдел nka бизнес детск питанияформирован...,0.0,...,academic ielts 80,1.0,1.0,1.0,3.785271,0.0,1.0,2.0,1.0,0.0
1,-1.216159,-0.032825,-1.144948,2.0,1.0,0.0,-0.974429,доработк оптимизац сqлзапрос сбор витрин дан,работ big datapythonsqldbt yandex datalens exc...,0.0,...,имеет оп создан андроидприложен kotlin вебсерв...,1.0,1.0,0.0,-0.692166,0.0,1.0,1.0,1.0,0.0
2,-0.593374,-0.032825,0.104330,2.0,1.0,0.0,0.350588,работ аналитическ отчетностьюосновн этап работ...,администрирован систем управлен звонкамиразраб...,0.0,...,ключев знан навыкикачествен анализ дан разрабо...,1.0,1.0,1.0,-0.692166,0.0,0.0,1.0,1.0,1.0
3,-0.375894,-0.032825,-0.557052,3.0,1.0,0.0,-0.643175,девелопмент техподдержк bi сист приложен платф...,расчет автоматизац расчет заработн плат прем л...,0.0,...,энергичн быстр обучаем открыт хобб спорт путеш...,0.0,1.0,1.0,0.427193,0.0,1.0,2.0,1.0,0.0
4,-1.067877,-0.032825,-0.924487,2.0,1.0,0.0,-0.643175,проведен исследован отток клиент создан витрин...,выполнен adhoc запрос руководств подготовк отч...,0.0,...,отвеча мессенджер почтуинтерес анализ дан им о...,1.0,1.0,1.0,1.546553,0.0,1.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9557,0.167807,-0.032825,-0.263104,0.0,1.0,0.0,0.350588,написан бизнестребован согласн поставлен задач...,формирован ежедневн ежемесячн ежеквартальн отч...,0.0,...,семейн положен женат дет жела график работ 52,0.0,0.0,0.0,-0.692166,0.0,1.0,1.0,0.0,0.0
9558,0.909217,-0.032825,-0.263104,1.0,1.0,0.0,-1.305683,анализ сведен,NaN,0.0,...,коммуникабельн стрессоустойчив работоспособн,0.0,0.0,0.0,-0.692166,0.0,1.0,1.0,0.0,0.0
9559,-0.148528,-0.032825,-0.263104,1.0,1.0,0.0,0.350588,обязанностиобработк входя звонк сообщен кандид...,мо обязан вход ежедневн обработк обзвон отклик...,0.0,...,коммуникабельн гибк целеустремлен исполнительн...,0.0,0.0,0.0,-0.692166,0.0,0.0,1.0,0.0,0.0
9560,1.156354,-0.032825,0.471765,1.0,1.0,0.0,1.013097,• подготовк отчетн мсфо иностра компан юрисдик...,организац работ подготовк управленческ отчетн ...,0.0,...,зна участк работ универсальн коммерческ банк в...,0.0,0.0,0.0,0.427193,0.0,1.0,2.0,0.0,0.0


# Preparing Data Text - Unbalanced

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

In [ ]:
def concating_features(features, df):
  sen_w_feats = []
  df = df.fillna("")

  for (i, row) in df.iterrows():
    combined = ''

    for col_name in features:
      combined += (str(row[col_name]))
    
    sen_w_feats.append(combined)

  df['text'] = sen_w_feats
  df = df.drop(features, axis=1)
  return df

In [ ]:
X = data_df.drop(['DataAnalyst', 'DataScientist'], axis=1)
y = data_df[['DataAnalyst', 'DataScientist']]

In [ ]:
text_features = ['Description', 'LastWorkDesc', 'LastWorkDesc2']

In [ ]:
data_full = concating_features(text_features, data_df)

In [ ]:
X = concating_features(text_features, X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

In [ ]:
tokenizer.fit_on_texts(X_train['text'].values)

In [ ]:
X_train_text = tokenizer.texts_to_sequences(X_train['text'].values)
X_train_text = tf.keras.utils.pad_sequences(X_train_text, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train_text.shape)

Shape of data tensor: (7171, 250)


In [ ]:
X_test_text = tokenizer.texts_to_sequences(X_test['text'].values)
X_test_text = tf.keras.utils.pad_sequences(X_test_text, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test_text.shape)

Shape of data tensor: (2391, 250)


In [ ]:
class_weights_da = class_weight.compute_class_weight(class_weight='balanced', classes = np.unique(y_train['DataAnalyst'].values), y=y_train['DataAnalyst'].values)
class_weights_ds = class_weight.compute_class_weight(class_weight='balanced', classes = np.unique(y_train['DataScientist'].values), y=y_train['DataScientist'].values)

In [ ]:
y_da_train = tf.convert_to_tensor(y_train['DataAnalyst'].values, dtype=tf.int64)
y_ds_train = tf.convert_to_tensor(y_train['DataScientist'].values)

y_da_test = tf.convert_to_tensor(y_test['DataAnalyst'].values, dtype=tf.int64)
y_ds_test = tf.convert_to_tensor(y_test['DataScientist'].values)

In [ ]:
X_num_train = tf.convert_to_tensor(X_train.drop(['text'], axis=1).values, dtype=tf.float64)
X_num_test = tf.convert_to_tensor(X_test.drop(['text'], axis=1).values, dtype=tf.float64)

# Undersampling

In [ ]:
df_da_1 = data_full[data_full['DataAnalyst']==1]
df_da_0 = data_full[data_full['DataAnalyst']==0]
df_da_0 = df_da_0.sample(len(df_da_1))

df_da_undersampling = df_da_0.append(df_da_1)
df_da_undersampling = df_da_undersampling.sample(frac=1).reset_index(drop=True)
df_da_undersampling = df_da_undersampling.drop(['DataScientist'], axis=1)

X_da_undersampling, y_da_undersampling = df_da_undersampling.drop(['DataAnalyst'], axis=1), df_da_undersampling['DataAnalyst']

<ipython-input-19-b79e93d79252>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_da_undersampling = df_da_0.append(df_da_1)


In [ ]:
df_ds_1 = data_full[data_full['DataScientist']==1]
df_ds_0 = data_full[data_full['DataScientist']==0]
df_ds_0 = df_ds_0.sample(len(df_ds_1))

df_ds_undersampling = df_ds_0.append(df_ds_1)
df_ds_undersampling = df_ds_undersampling.sample(frac=1).reset_index(drop=True)
df_ds_undersampling = df_ds_undersampling.drop(['DataAnalyst'], axis=1)

X_ds_undersampling, y_ds_undersampling = df_ds_undersampling.drop(['DataScientist'], axis=1), df_ds_undersampling['DataScientist']

<ipython-input-20-59113de0f258>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ds_undersampling = df_ds_0.append(df_ds_1)


In [ ]:
X_full_text = tokenizer.texts_to_sequences(data_full['text'].values)
X_full_text = tf.keras.utils.pad_sequences(X_full_text, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_full_text.shape)

Shape of data tensor: (9562, 250)


In [ ]:
X_da_text_under = tokenizer.texts_to_sequences(X_da_undersampling['text'].values)
X_da_text_under = tf.keras.utils.pad_sequences(X_da_text_under, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_da_text_under.shape)

Shape of data tensor: (1374, 250)


In [ ]:
X_ds_text_under = tokenizer.texts_to_sequences(X_ds_undersampling['text'].values)
X_ds_text_under = tf.keras.utils.pad_sequences(X_ds_text_under, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_ds_text_under.shape)

Shape of data tensor: (222, 250)


In [ ]:
y_da_undersampling = tf.convert_to_tensor(y_da_undersampling, dtype=tf.int64)
y_ds_undersampling = tf.convert_to_tensor(y_ds_undersampling, dtype=tf.int64)

y_full_da = tf.convert_to_tensor(data_full['DataAnalyst'].values, dtype=tf.int64)
y_full_ds = tf.convert_to_tensor(data_full['DataScientist'].values)

In [ ]:
X_da_num_under = tf.convert_to_tensor(X_da_undersampling.drop(['text'], axis=1).values, dtype=tf.float64)
X_ds_num_under = tf.convert_to_tensor(X_ds_undersampling.drop(['text'], axis=1).values, dtype=tf.float64)
X_num_full = tf.convert_to_tensor(data_full.drop(['text', 'DataAnalyst', 'DataScientist'], axis=1).values, dtype=tf.float64)

# Model

In [ ]:

nlp_input = Input(shape=(250, )) 
meta_input = Input(shape=(21,))
emb = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_text.shape[1])(nlp_input) 
nlp_out = Bidirectional(LSTM(128))(emb) 
concat = tf.keras.layers.Concatenate(axis=1)([nlp_out, meta_input]) 
classifier = Dense(32, activation='relu')(concat) 
output = Dense(1, activation='sigmoid')(classifier) 
model = Model(inputs=[nlp_input , meta_input], outputs=[output])

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=METRICS)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 250)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 250, 100)     5000000     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 256)          234496      ['embedding[0][0]']              
                                                                                                  
 input_2 (InputLayer)           [(None, 21)]         0           []                               
                                                                                              

## Unbalanced

In [ ]:
model.fit([X_train_text, X_num_train], y_da_train, epochs=10)

Epoch 1/10
225/225 [==============================] - 57s 209ms/step - loss: 0.2280 - accuracy: 0.9227 - precision: 0.3394 - recall: 0.1118
Epoch 2/10
225/225 [==============================] - 36s 159ms/step - loss: 0.1254 - accuracy: 0.9516 - precision: 0.7070 - recall: 0.5250
Epoch 3/10
225/225 [==============================] - 25s 110ms/step - loss: 0.0620 - accuracy: 0.9806 - precision: 0.8709 - recall: 0.8483
Epoch 4/10
225/225 [==============================] - 18s 80ms/step - loss: 0.0258 - accuracy: 0.9927 - precision: 0.9648 - recall: 0.9301
Epoch 5/10
225/225 [==============================] - 14s 61ms/step - loss: 0.0095 - accuracy: 0.9969 - precision: 0.9800 - recall: 0.9760
Epoch 6/10
225/225 [==============================] - 14s 61ms/step - loss: 0.0053 - accuracy: 0.9983 - precision: 0.9861 - recall: 0.9900
Epoch 7/10
225/225 [==============================] - 12s 54ms/step - loss: 0.0015 - accuracy: 0.9997 - precision: 0.9960 - recall: 1.0000
Epoch 8/10
225/225 [====

In [ ]:
y_pred_da = model.predict([X_test_text, X_num_test])

75/75 [==============================] - 1s 10ms/step


In [ ]:
y_pred_da = y_pred_da.flatten()

In [ ]:
check_da = np.where(y_pred_da > 0.5, 1, 0)

In [ ]:
accuracy_score(check_da, y_da_test)

0.9268088665830196

In [ ]:
f1_score(check_da, y_da_test)

0.3346007604562738

In [ ]:
print(precision_score(check_da, y_da_test))
print(recall_score(check_da,y_da_test ))

0.23655913978494625
0.5714285714285714


In [ ]:
model.fit([X_train_text, X_num_train], y_ds_train, epochs=10)

Epoch 1/10
225/225 [==============================] - 9s 31ms/step - loss: 0.0622 - accuracy: 0.9866 - precision: 0.1111 - recall: 0.0244
Epoch 2/10
225/225 [==============================] - 7s 32ms/step - loss: 0.0257 - accuracy: 0.9916 - precision: 0.7292 - recall: 0.4268
Epoch 3/10
225/225 [==============================] - 8s 35ms/step - loss: 0.0093 - accuracy: 0.9974 - precision: 0.9565 - recall: 0.8049
Epoch 4/10
225/225 [==============================] - 8s 36ms/step - loss: 0.0050 - accuracy: 0.9989 - precision: 0.9744 - recall: 0.9268
Epoch 5/10
225/225 [==============================] - 8s 34ms/step - loss: 0.0017 - accuracy: 0.9996 - precision: 0.9877 - recall: 0.9756
Epoch 6/10
225/225 [==============================] - 6s 28ms/step - loss: 6.7008e-04 - accuracy: 0.9997 - precision: 0.9878 - recall: 0.9878
Epoch 7/10
225/225 [==============================] - 8s 33ms/step - loss: 7.4024e-04 - accuracy: 0.9997 - precision: 0.9878 - recall: 0.9878
Epoch 8/10
225/225 [======

In [ ]:
y_pred_ds = model.predict([X_test_text, X_num_test])
y_pred_ds = y_pred_da.flatten()

75/75 [==============================] - 1s 10ms/step


In [ ]:
check_ds = np.where(y_pred_da > 0.5, 1, 0)

In [ ]:
accuracy_score(check_ds, y_ds_test)

0.9657047260560435

In [ ]:
f1_score(check_ds, y_ds_test)

0.22641509433962265

In [ ]:
print(precision_score(check_ds, y_ds_test))
print(recall_score(check_ds,y_ds_test ))

0.41379310344827586
0.15584415584415584


## Undersampling

In [ ]:
model.fit([X_da_text_under, X_da_num_under], y_da_undersampling, epochs=10)

Epoch 1/10
43/43 [==============================] - 1s 30ms/step - loss: 0.6683 - accuracy: 0.8028 - precision: 0.8240 - recall: 0.7700
Epoch 2/10
43/43 [==============================] - 1s 26ms/step - loss: 0.2016 - accuracy: 0.9345 - precision: 0.9370 - recall: 0.9316
Epoch 3/10
43/43 [==============================] - 2s 36ms/step - loss: 0.0849 - accuracy: 0.9774 - precision: 0.9838 - recall: 0.9709
Epoch 4/10
43/43 [==============================] - 1s 21ms/step - loss: 0.0418 - accuracy: 0.9898 - precision: 0.9884 - recall: 0.9913
Epoch 5/10
43/43 [==============================] - 1s 21ms/step - loss: 0.0222 - accuracy: 0.9942 - precision: 0.9971 - recall: 0.9913
Epoch 6/10
43/43 [==============================] - 1s 26ms/step - loss: 0.0090 - accuracy: 0.9971 - precision: 0.9971 - recall: 0.9971
Epoch 7/10
43/43 [==============================] - 1s 31ms/step - loss: 0.0035 - accuracy: 0.9985 - precision: 0.9985 - recall: 0.9985
Epoch 8/10
43/43 [==============================

In [ ]:
y_pred_da_under = model.predict([X_full_text, X_num_full])

299/299 [==============================] - 3s 8ms/step


In [ ]:
y_pred_da_under = y_pred_da_under.flatten()
check_da_under = np.where(y_pred_da_under > 0.5, 1, 0)

In [ ]:
accuracy_score(check_da_under, y_full_da)

0.8985567872829952

In [ ]:
f1_score(check_da_under, y_full_da)

0.5858240819812126

In [ ]:
print(precision_score(check_da_under, y_full_da))
print(recall_score(check_da_under, y_full_da))

0.9985443959243085
0.41450151057401813


In [ ]:
model.fit([X_ds_text_under, X_ds_num_under], y_ds_undersampling, epochs=10)

Epoch 1/10
7/7 [==============================] - 0s 30ms/step - loss: 0.3174 - accuracy: 0.9414 - precision: 0.9224 - recall: 0.9640
Epoch 2/10
7/7 [==============================] - 0s 28ms/step - loss: 0.0758 - accuracy: 0.9685 - precision: 0.9727 - recall: 0.9640
Epoch 3/10
7/7 [==============================] - 0s 32ms/step - loss: 0.0220 - accuracy: 0.9910 - precision: 1.0000 - recall: 0.9820
Epoch 4/10
7/7 [==============================] - 1s 143ms/step - loss: 0.0097 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 5/10
7/7 [==============================] - 0s 78ms/step - loss: 0.0069 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 6/10
7/7 [==============================] - 0s 22ms/step - loss: 0.0049 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 7/10
7/7 [==============================] - 0s 57ms/step - loss: 0.0033 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 8/10
7/7 [==============================] - 0s 23ms/ste

In [ ]:
y_pred_ds_under = model.predict([X_full_text, X_num_full])

299/299 [==============================] - 3s 8ms/step


In [ ]:
y_pred_ds_under = y_pred_ds_under.flatten()
check_ds_under = np.where(y_pred_ds_under > 0.5, 1, 0)

In [ ]:
accuracy_score(check_ds_under, y_full_ds)

0.9188454298263962

In [ ]:
f1_score(check_ds_under, y_full_ds)

0.22244488977955915

In [ ]:
print(precision_score(check_ds_under, y_full_ds))
print(recall_score(check_ds_under, y_full_ds))

1.0
0.12514092446448705


## Oversampling

In [ ]:
X_vectorized = np.concatenate((X_train_text, X_train.drop(['text'], axis=1).values), axis=1)

In [ ]:
X_vectorized.shape

(7171, 271)

In [ ]:
sm = SMOTE(random_state=42)
X_da_oversampling, y_da_oversampling = sm.fit_resample(X_vectorized, y_da_train)
X_ds_oversampling, y_ds_oversampling = sm.fit_resample(X_vectorized, y_ds_train)

In [ ]:
X_da_over_text = np.array([ele[:-21] for ele in X_da_oversampling])
X_ds_over_text = np.array([ele[:-21] for ele in X_ds_oversampling])

X_da_over_num = np.array([ele[250:] for ele in X_da_oversampling])
X_ds_over_num = np.array([ele[250:] for ele in X_ds_oversampling])

In [ ]:
model.fit([X_da_over_text, X_da_over_num], y_da_oversampling, epochs=10)

Epoch 1/10
417/417 [==============================] - 18s 39ms/step - loss: 0.3631 - accuracy: 0.8489 - precision: 0.8542 - recall: 0.8414
Epoch 2/10
417/417 [==============================] - 15s 37ms/step - loss: 0.1332 - accuracy: 0.9516 - precision: 0.9439 - recall: 0.9604
Epoch 3/10
417/417 [==============================] - 12s 29ms/step - loss: 0.0419 - accuracy: 0.9879 - precision: 0.9855 - recall: 0.9904
Epoch 4/10
417/417 [==============================] - 13s 31ms/step - loss: 0.0144 - accuracy: 0.9961 - precision: 0.9952 - recall: 0.9970
Epoch 5/10
417/417 [==============================] - 13s 31ms/step - loss: 0.0069 - accuracy: 0.9990 - precision: 0.9991 - recall: 0.9988
Epoch 6/10
417/417 [==============================] - 11s 27ms/step - loss: 0.0021 - accuracy: 0.9999 - precision: 0.9999 - recall: 0.9999
Epoch 7/10
417/417 [==============================] - 12s 28ms/step - loss: 3.2929e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 8/10
417/417 [===

In [ ]:
y_pred_da_over = model.predict([X_test_text, X_num_test])
y_pred_da_over = y_pred_da_over.flatten()
check_da_over = np.where(y_pred_da_over > 0.5, 1, 0)

75/75 [==============================] - 1s 10ms/step


In [ ]:
check_da_over

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
accuracy_score(check_da_over, y_da_test)

0.8578000836470097

In [ ]:
f1_score(check_da_over, y_da_test)

0.43521594684385384

In [ ]:
print(precision_score(check_da_over, y_da_test))
print(recall_score(check_da_over, y_da_test))

0.7043010752688172
0.31490384615384615


In [ ]:
model.fit([X_ds_over_text, X_ds_over_num], y_ds_oversampling, epochs=10)

Epoch 1/10
444/444 [==============================] - 18s 36ms/step - loss: 0.0789 - accuracy: 0.9740 - precision: 0.9734 - recall: 0.9746
Epoch 2/10
444/444 [==============================] - 15s 34ms/step - loss: 0.0114 - accuracy: 0.9959 - precision: 0.9976 - recall: 0.9942
Epoch 3/10
444/444 [==============================] - 13s 30ms/step - loss: 0.0031 - accuracy: 0.9991 - precision: 0.9993 - recall: 0.9989
Epoch 4/10
444/444 [==============================] - 14s 31ms/step - loss: 8.9565e-04 - accuracy: 0.9998 - precision: 1.0000 - recall: 0.9996
Epoch 5/10
444/444 [==============================] - 14s 31ms/step - loss: 3.9767e-04 - accuracy: 0.9999 - precision: 0.9999 - recall: 0.9999
Epoch 6/10
444/444 [==============================] - 13s 29ms/step - loss: 1.7953e-04 - accuracy: 0.9999 - precision: 0.9999 - recall: 1.0000
Epoch 7/10
444/444 [==============================] - 13s 28ms/step - loss: 1.3324e-04 - accuracy: 0.9999 - precision: 1.0000 - recall: 0.9999
Epoch 8/10


In [ ]:
y_pred_ds_over = model.predict([X_test_text, X_num_test])
y_pred_ds_over = y_pred_ds_over.flatten()
check_ds_over = np.where(y_pred_ds_over > 0.5, 1, 0)

75/75 [==============================] - 1s 8ms/step


In [ ]:
accuracy_score(check_ds_over, y_ds_test)

0.9167712254286909

In [ ]:
f1_score(check_ds_over, y_ds_test)

0.11555555555555556

In [ ]:
print(precision_score(check_ds_over, y_ds_test))
print(recall_score(check_ds_over, y_ds_test))

0.4482758620689655
0.0663265306122449
